# Geotiff Interpolation Tutorial

The purpose of this notebook is to demonstrate interpolation from geotiff files to lon/lat pairs. In this case, the geotiff files are bands from HRRR grib2 files collected using `wrfxpy` methods on Alderaan. The lon/lat pairs are collected from RAWS stations with `Mesopy`, and assumed to be in the WGS84 standard coordinate system AKA EPSG 4326.

The tiff files are saved with naming a convention that stores the UTC time info as well as the associated band. See:

https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf00.grib2.shtml

# Setup 

In [ ]:
import os
import os.path as osp
import numpy as np
from osgeo import gdal, osr
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
path = os.getcwd()
tpath = osp.join(path,"hrrr.t00z.wrfprsf00.616.tif")

In [ ]:
# Get RAWS Station lat/lon
sts = pd.read_csv("C:/Users/jhirs/Documents/Projects/openwfm/notebooks/fmda/data/raws_stations_CO.csv")
sts

In [ ]:
# Extract data from tif file
ds = gdal.Open(tpath)
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
gp = ds.GetProjection()
# data = np.array(ds.ReadAsArray())

In [ ]:
print(width)
print(height)
print(gt)
print(gp)

In [ ]:
print('Raster count: ' + str(ds.RasterCount))

In [ ]:
band = ds.GetRasterBand(1)
data = band.ReadAsArray()

# Plot Raster File

Using `imshow`, add a point at 100,100 just to demonstrate image indexing.

source: https://www.geeksforgeeks.org/visualizing-tiff-file-using-matplotlib-and-gdal-using-python/

In [ ]:
# Plot
plt.imshow(data)
plt.plot(100, 100, marker='o', color='blue', markersize=6)
plt.annotate("(100,100)", (100,100), color='blue')

# Interp Lat/Lon

Source (nearest neighbor method): https://stackoverflow.com/questions/69034965/given-a-geotiff-file-how-does-one-find-the-single-pixel-closest-to-a-given-lati

In [ ]:
point_srs = osr.SpatialReference()
point_srs.ImportFromEPSG(4326) # hardcode for lon/lat

# GDAL>=3: make sure it's x/y
# see https://trac.osgeo.org/gdal/wiki/rfc73_proj6_wkt2_srsbarn
point_srs.SetAxisMappingStrategy(osr.OAMS_TRADITIONAL_GIS_ORDER)     

file_srs = osr.SpatialReference()
file_srs.ImportFromWkt(gp)

In [ ]:
ct = osr.CoordinateTransformation(point_srs, file_srs)

point_x = sts['lon'][0] # lon
point_y = sts['lat'][0]  # lat
mapx, mapy, z = ct.TransformPoint(point_x, point_y) # output: coordinate pair (m)

In [ ]:
gt_inv = gdal.InvGeoTransform(gt)
pixel_x, pixel_y = gdal.ApplyGeoTransform(gt_inv, mapx, mapy)

We plot the image with the pixel annotated. The lon/lat pair is from a RAWS station near Colorado Springs, which matches the image below.

In [ ]:
plt.imshow(data)

# Plot pixel translation with annotations
plt.plot(pixel_x,pixel_y, marker='o', color='blue', markersize=6)
plt.annotate(f"Pixels: ({round(pixel_x, 2)}, {round(pixel_y, 2)})", xy=(pixel_x, pixel_y),
            xytext=(pixel_x-100, pixel_y-100), fontsize=8, color='blue')
plt.annotate(f"Lon/Lat: ({round(point_x, 2)}, {round(point_y, 2)})", xy=(pixel_x, pixel_y),
           xytext=(pixel_x-100, pixel_y-50), fontsize=8, color='blue')

After this point, the tutorial goes on to describe a method for nearest neighbor. This is just one form of interpolation, so various methods will be explored below.

We will plot a zoomed in version of the pixels to demonstrate this.

In [ ]:
plt.imshow(data)

# Plot pixel translation with annotations
plt.plot(pixel_x,pixel_y, marker='o', color='blue', markersize=6,
        label=f"({round(pixel_x, 2)}, {round(pixel_y, 2)})")

# Zoom in, set limits from plotted pixel
offset = 5
plt.xlim(pixel_x-offset, pixel_x+offset) # Note different +/- bc origin is topright
plt.ylim(pixel_y+offset,pixel_y-offset)

# Plot 4 points bracketing target pixel
x1, y1=np.floor(pixel_x), np.floor(pixel_y)
x2, y2=np.floor(pixel_x), np.ceil(pixel_y)
x3, y3=np.ceil(pixel_x), np.floor(pixel_y)
x4, y4=np.ceil(pixel_x), np.ceil(pixel_y)

plt.plot(x1,y1, marker='o', color='red', markersize=4, label='Bracketing Pixels')
plt.plot(x2,y2, marker='o', color='red', markersize=4)
plt.plot(x3,y3, marker='o', color='red', markersize=4)
plt.plot(x4,y4, marker='o', color='red', markersize=4)

plt.legend()

# Interpolation Methods

## Nearest Neighbor (L1)

The tutorial linked above simply rounds the pixel x and y coordinates to the nearest pixels and takes the value from that grid location. This is mathematically equivalent to an L1 nearest neighbor, or manhattan distance minimization.

In [ ]:
# round to pixel
x_l1 = round(pixel_x)
y_l1 = round(pixel_y)

In [ ]:
plt.imshow(data)

# Plot pixel translation with annotations
plt.plot(pixel_x,pixel_y, marker='o', color='blue', markersize=6,
        label=f"({round(pixel_x, 2)}, {round(pixel_y, 2)})")

# Zoom in, set limits from plotted pixel
offset = 5
plt.xlim(pixel_x-offset, pixel_x+offset) # Note different +/- bc origin is topright
plt.ylim(pixel_y+offset,pixel_y-offset)

# Plot 4 points bracketing target pixel
x1, y1=np.floor(pixel_x), np.floor(pixel_y)
x2, y2=np.floor(pixel_x), np.ceil(pixel_y)
x3, y3=np.ceil(pixel_x), np.floor(pixel_y)
x4, y4=np.ceil(pixel_x), np.ceil(pixel_y)

plt.plot(x1,y1, marker='o', color='red', markersize=4, label='Bracketing Pixels')
plt.plot(x2,y2, marker='o', color='red', markersize=4)
plt.plot(x3,y3, marker='o', color='red', markersize=4)
plt.plot(x4,y4, marker='o', color='red', markersize=4)

# Plot interpolated pixel
plt.plot(x_l1,y_l1, marker='o', color='purple', markersize=5,
        label='Interpolated')
interp_val = data[y_l1, x_l1]
plt.annotate("Interp. Value="+str(round(interp_val, 5)), xy=(x_l1, y_l1), xytext=(x_l1-1, y_l1-2), color='purple')

plt.title("L1 Nearest Neighbor")
plt.legend()

NOTE: The HRRR documentation says that 2m Temperature, band 616, should be in degrees K. But obviously this data is in degrees C. We will have to check this in the future.

## Nearest Neighbor (L2)

In `wrfxpy`, the function `find_closest_grid_point` is defined to find the L2 nearest neighbor, which finds the minimum sum of squared distance (Euclidean norm).

https://github.com/openwfm/wrfxpy/blob/master/src/utils.py#L529

NOTE: very slow implementation, but I wanted to reproduce the method clearly. In this case, the interpolated value is the same as L1.

In [ ]:
x = np.arange(0, band.XSize)
y = np.arange(0, band.YSize)
pixels = [(xx, yy) for xx in x for yy in y]
d = np.zeros(len(pixels))
for i in range(0, len(pixels)):
    p = pixels[i]
    d[i] = (pixel_x - p[0])**2 + (pixel_y - p[1])**2

nearest = pixels[np.argmin(d)]

In [ ]:
plt.imshow(data)

# Plot pixel translation with annotations
plt.plot(pixel_x,pixel_y, marker='o', color='blue', markersize=6,
        label=f"({round(pixel_x, 2)}, {round(pixel_y, 2)})")

# Zoom in, set limits from plotted pixel
offset = 5
plt.xlim(pixel_x-offset, pixel_x+offset) # Note different +/- bc origin is topright
plt.ylim(pixel_y+offset,pixel_y-offset)

# Plot 4 points bracketing target pixel
x1, y1=np.floor(pixel_x), np.floor(pixel_y)
x2, y2=np.floor(pixel_x), np.ceil(pixel_y)
x3, y3=np.ceil(pixel_x), np.floor(pixel_y)
x4, y4=np.ceil(pixel_x), np.ceil(pixel_y)

plt.plot(x1,y1, marker='o', color='red', markersize=4, label='Bracketing Pixels')
plt.plot(x2,y2, marker='o', color='red', markersize=4)
plt.plot(x3,y3, marker='o', color='red', markersize=4)
plt.plot(x4,y4, marker='o', color='red', markersize=4)

# find nearest L2 pixel
plt.plot(nearest[0],nearest[1], marker='o', color='purple', markersize=5,
        label='Interpolated')
interp_val = data[nearest[1], nearest[0]]
plt.annotate("Interp. Value="+str(round(interp_val, 5)), xy=(x_l1, y_l1), xytext=(x_l1-1, y_l1-2), color='purple')


plt.title("L2 Nearest Neighbor")
plt.legend()

## Average Bracketing Points

Given the 4 bracketing points, interpolate the mean value.

In [ ]:
plt.imshow(data)

# Plot pixel translation with annotations
plt.plot(pixel_x,pixel_y, marker='o', color='blue', markersize=6,
        label=f"({round(pixel_x, 2)}, {round(pixel_y, 2)})")

# Zoom in, set limits from plotted pixel
offset = 5
plt.xlim(pixel_x-offset, pixel_x+offset) # Note different +/- bc origin is topright
plt.ylim(pixel_y+offset,pixel_y-offset)

# Plot 4 points bracketing target pixel
x1, y1=int(np.floor(pixel_x)), int(np.floor(pixel_y))
x2, y2=int(np.floor(pixel_x)), int(np.ceil(pixel_y))
x3, y3=int(np.ceil(pixel_x)), int(np.floor(pixel_y))
x4, y4=int(np.ceil(pixel_x)), int(np.ceil(pixel_y))

plt.plot(x1,y1, marker='o', color='red', markersize=4, label='Bracketing Pixels')
plt.plot(x2,y2, marker='o', color='red', markersize=4)
plt.plot(x3,y3, marker='o', color='red', markersize=4)
plt.plot(x4,y4, marker='o', color='red', markersize=4)

interp_val = np.mean([data[y1, x1], data[y2, x2], data[y3, x3], data[y4, 4]])
plt.annotate("Interp. Value="+str(round(interp_val, 5)), xy=(pixel_x, pixel_y), xytext=(pixel_x-1, pixel_y-2), color='purple')


plt.title("Average 4 Nearest Neighbors")
plt.legend()

## Scipy griddata interpolation package

https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html

In [ ]:
from scipy.interpolate import griddata

## Validate Geolocation

Goals:

* create two fake geotiff
* create data array with known lon/lat values
* evaluate and you should get coord back exactly

Steps here:

* Convert pixel indices (e.g. (0,0), (0,1)...) to lat lon
* Save lats and lons in array of same dimension as raster band
* Save as geotiff
* Read that file back in to do steps above


Modified from Source: https://stackoverflow.com/questions/59052516/find-lat-long-coordinates-from-pixel-point-in-geotiff-using-python-and-gdaltly

In [ ]:
# Generate Arrays of pixel indices
lons = np.zeros((1059, 1799))
lats = np.zeros((1059, 1799))

In [ ]:
from osgeo import osr, ogr, gdal

def pixel_to_world(geo_matrix, x, y):
    # Given geotransform info of a geotiff file and an (x,y) pixel coord pair, return the coord pair that matches the geotiff in meters
    # Inputs: 
    # geomatrix: output of ds.GetGeoTransform() for given geotiff file
    # tuple of length 6 contains: 
    # A geotransform consists in a set of 6 coefficients
    # GT(0) x-coordinate of the upper-left corner of the upper-left pixel.
    # GT(1) w-e pixel resolution / pixel width.
    # GT(2) row rotation (typically zero).
    # GT(3) y-coordinate of the upper-left corner of the upper-left pixel.
    # GT(4) column rotation (typically zero).
    # GT(5) n-s pixel resolution / pixel height (negative value for a north-up image).
    # x: pixel index x coord (1)
    # y: pixel index y coord (1)
    # Return: coordinates of same point as given x,y as offset from UL (m)
    # Example: pixel_to_world(mat, 0, 0) returns UL x,y from geotiff
    
    ul_x = geo_matrix[0]
    ul_y = geo_matrix[3]
    x_dist = geo_matrix[1]
    y_dist = geo_matrix[5]
    _x = x * x_dist + ul_x
    _y = y * y_dist + ul_y
    return _x, _y


def build_transform_inverse(dataset, EPSG):
    # Given gdal dataset and target EPSG, return transformation function that transforms meter coord pairs to pixel coord pairs 
    # Inputs:
    # dataset: geotiff file
    # EPSG: integer
    source = osr.SpatialReference(wkt=dataset.GetProjection())
    target = osr.SpatialReference()
    target.ImportFromEPSG(EPSG)
    return osr.CoordinateTransformation(source, target)

def world_to_epsg(wx, wy, trans):
    # Inputs:
    # wx, wy: output of build_transform_inverse
    # wx: x coordinate (m) related to geotiff reference point
    # wy: y coordinate (m) related to geotiff reference point
    # transform: function to transform to given epsg, function type is osgeo.osr.CoordinateTransformation
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(wx, wy)
    point.Transform(trans)
    return point

def find_spatial_coordinate_from_pixel(dataset, x, y, epsg=4326):
    # Given gdal dataset, target x y pixel pair, and EPSG, return the EPSG defined coordinate pair 
    # dataset: gdal dataset, from geotiff file
    # x: pixel x coord
    # y: pixel y coord
    # Return: coord pair in given epsg, eg lat/lon
    transform = build_transform_inverse(ds, epsg)
    world_x, world_y = pixel_to_world(dataset.GetGeoTransform(), x, y)
    point = world_to_epsg(world_x, world_y, transform)
    return point.GetX(), point.GetY()

ds = gdal.Open(tpath)

We want this process to match information from the gdalinfo in the geotiff file. The command line `gdalinfo hrrr.t00z.wrfprsf00.616.tif` returns info on the corner coordinates and center (not (0,0) for some reason). The following output should thus match the return of that command.

`gdalinfo` returns coordinates in the "degrees minute second" format, so we need to convert decimal degrees to this format to make sure it matches. 


In [ ]:
## Test world_to_epsg
## For 'center' in file (-520.143, -306.153), not zero for some reason
## Should return ( 97d30'21.52"W, 38d29'50.09"N)
trans = build_transform_inverse(ds, 4326)
pt = world_to_epsg(-520.143, -306.153, trans)
print(f"({pt.GetX()},{pt.GetY()})")

In [ ]:
## Test on a corner coords
print(find_spatial_coordinate_from_pixel(ds, 0, 0)) # upper left
print(find_spatial_coordinate_from_pixel(ds, 0, 1058)) # upper right
print(find_spatial_coordinate_from_pixel(ds, 1798, 0)) # lower left
print(find_spatial_coordinate_from_pixel(ds, 1798, 1058)) # lower right

`gdalinfo` returns coordinates with the format "degree minute second(direction)", so we need to convert the previous decimal coordinates to make sure we are matching `gdalinfo`.